In [30]:
import os
import nengo
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, \
    Conv3D, MaxPooling2D, Conv1D, AveragePooling1D, LSTM
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold
from keras import backend as K
from sklearn.utils import shuffle

In [31]:
# Data from males is by default saved in dataset_result/men_only.npz
men_dataset_path = os.path.join('dataset_result', 'men_only.npz')
dataset = np.load(men_dataset_path)

features, labels = dataset['features'], dataset['labels']

print('Features shape:', features.shape)
print('Labels shape:', labels.shape)

Features shape: (1296, 14, 36, 10)
Labels shape: (1296,)


In [32]:
# Set seed for consistency
seed = 23
np.random.seed(seed)
tf.random.set_seed(seed)

In [33]:
# Reshape features for the NN
features = features.reshape((features.shape[0], 1, -1))
print('Features shape:', features.shape)

# Convert labels to one hot encoding
labels = labels.reshape(-1, 1)
labels = OneHotEncoder().fit_transform(labels).toarray()
labels = labels.reshape((labels.shape[0], 1, -1))
print('Labels shape:', labels.shape)

Features shape: (1296, 1, 5040)
Labels shape: (1296, 1, 2)


In [34]:
# Function to create the CNN model for classification
def cnn_model():
    inp = Input(shape=(14, 36, 10), name='input_layer')
    conv1 = Conv2D(filters=32, kernel_size=(5, 5), activation=tf.nn.relu)(inp)
    dropout1 = Dropout(0.2, seed=seed)(conv1)
    avg_pool1 = AveragePooling2D(pool_size=(2, 2))(dropout1)
    conv2 = Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu)(avg_pool1)
    dropout2 = Dropout(0.2, seed=seed)(conv2)
    avg_pool2 = AveragePooling2D(pool_size=(2, 2))(dropout2)
    flatten = Flatten()(avg_pool2)
    dense1 = Dense(512, activation=tf.nn.relu)(flatten)
    dropout3 = Dropout(0.2, seed=seed)(dense1)
    dense2 = Dense(256, activation=tf.nn.relu)(dropout3)
    output = Dense(2, activation=tf.nn.softmax, name='output_layer')(dense2)

    return Model(inputs=inp, outputs=output)

In [35]:
def run_ann(model, train, val, test,
            params_save_path,
            iteration,
            shuffle_training=True,
            num_epochs=30,
            minibatch_size=10,
            early_stopping=False):
    x_train, y_train = train[0], train[1]
    x_test, y_test = test[0], test[1]
    x_val, y_val = val[0], val[1]

    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=minibatch_size) as simulator:
        simulator.compile(optimizer=keras.optimizers.Adam(),
                          loss=keras.losses.BinaryCrossentropy(),
                          metrics=['accuracy'])

        input_layer = converter.inputs[model.get_layer('input_layer')]
        output_layer = converter.outputs[model.get_layer('output_layer')]

        simulator.fit(
            x={ input_layer: x_train }, y={ output_layer: y_train },
            validation_data=(
                { input_layer: x_val }, { output_layer: y_val }
            ),
            epochs=num_epochs,
            shuffle=shuffle_training,
            callbacks=[EarlyStopping(patience=8, verbose=1, restore_best_weights=True)] if early_stopping else None
            # early stop to avoid overfitting
        )

        simulator.save_params(params_save_path)
        ann_eval = simulator.evaluate(x={ input_layer: x_test }, y={ output_layer: y_test }) # evaluate accuracy
        print('{}. ann accuracy: {:5f}%'.format(iteration, ann_eval['probe_accuracy'] * 100)) # log accuracy
        return ann_eval['probe_accuracy'] # return accuracy


def run_snn(model, x_test, y_test, params_load_path, iteration, timesteps=50, scale_firing_rates=1000, synapse=0.01):
    converter = nengo_dl.Converter(
        model,
        swap_activations={ tf.nn.relu: nengo.SpikingRectifiedLinear() },
        scale_firing_rates=scale_firing_rates,
        synapse=synapse
    ) # create a Nengo converter object and swap all relu activations with spiking relu

    with converter.net:
        nengo_dl.configure_settings(stateful=False)

    input_layer = converter.inputs[model.get_layer('input_layer')] # input layer for simulator
    output_layer = converter.outputs[model.get_layer('output_layer')] # output layer for simulator

    x_test_tiled = np.tile(x_test, (1, timesteps, 1)) # tile test data to timesteps

    with nengo_dl.Simulator(converter.net) as simulator:
        simulator.load_params(params_load_path)

        predictions = simulator.predict({ input_layer: x_test_tiled })[output_layer] # get results from prediction
        predictions = predictions[:,-1,:] # get last time step

        predictions = np.argmax(predictions, axis=-1) # get argmax
        y_test = np.squeeze(y_test, axis=1) # remove time dimension from labels since its not relevant
        y_test = np.argmax(y_test, axis=-1) # get argmax of y test as well for comparison

        snn_acc = (predictions == y_test).mean()

    print('{}. SNN accuracy: {:5f}%'.format(
            iteration, snn_acc * 100
        ))

    return snn_acc


In [36]:
os.makedirs('male_nengo_params', exist_ok=True)
ann, snn = [], []

num_splits = 10
iteration = 1

features, labels = shuffle(features, labels, random_state=seed)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, shuffle=False)


for train, val in KFold(n_splits=num_splits).split(x_train):
    x_train_curr, y_train_curr = x_train[train], y_train[train]
    x_val, y_val = x_train[val], y_train[val]

    print(x_test.shape)

    params_path = os.path.join('male_nengo_params', 'params_{}'.format(iteration))
    model = cnn_model()

    # run ann
    ann_result = run_ann(model=model,
                      train=(x_train_curr, y_train_curr),
                      val=(x_val, y_val),
                      test=(x_test, y_test),
                      params_save_path=params_path,
                      iteration=iteration,
                      early_stopping=True,
                      num_epochs=50)

    # run snn
    snn_result = run_snn(model=model,
                         x_test=x_test,
                         y_test=y_test,
                         params_load_path=params_path,
                         iteration=iteration)

    ann.append(ann_result)
    snn.append(snn_result)
    iteration += 1

    K.clear_session()
    del model

(130, 1, 5040)
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/50
104/104 [==============================] - 5s 17ms/step - loss: 0.8261 - probe_loss: 0.8261 - probe_accuracy: 0.5209 - val_loss: 0.6965 - val_probe_loss: 0.6965 - val_probe_accuracy: 0.4727

c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:588: UserWarning: Activation type <function softmax_v2 at 0x000002095DAAC940> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\simulator.py:1921: UserWarning: Number of elements in input data (117) is not evenly divisible by Simulator.minibatch_size (10); input data will be truncated.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\simulator.py:1921: UserWarning: Number of elements in input data (1049) is not evenly divisible by Simulator.minibatch_size (10); input data will be truncated.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\simulator.py:1921: UserWarning: Number of

In [37]:
print(ann)
print(snn)


print('Max ANN:', max(ann))
print('Max SNN:', max(ann))
print('Avg ANN:', np.average(ann))
print('Avg SNN:', np.average(snn))

[0.5461538434028625, 0.446153849363327, 0.4769230782985687, 0.5461538434028625, 0.5538461804389954, 0.5538461804389954, 0.5461538434028625, 0.5076923370361328, 0.5461538434028625, 0.446153849363327]
[0.5307692307692308, 0.4307692307692308, 0.45384615384615384, 0.43846153846153846, 0.3923076923076923, 0.5692307692307692, 0.5153846153846153, 0.5615384615384615, 0.5461538461538461, 0.4461538461538462]
Max ANN: 0.5538461804389954
Max SNN: 0.5538461804389954
Avg ANN: 0.5169230848550797
Avg SNN: 0.4884615384615385
